In [ ]:
import zipfile
import os

zip_path = '/content/Daily Historical Stock Prices (1970 - 2018).zip'

# Check if it's a valid zip file
if zipfile.is_zipfile(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('.')
        print("Zip file extracted.")
else:
    print("Error: The file is not a valid zip file.")


Error: The file is not a valid zip file.


In [ ]:
# Reload and initialize the datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Load the datasets
historical_stocks = pd.read_csv('historical_stocks.csv')
historical_stock_prices = pd.read_csv('historical_stock_prices.csv')

print("Dataset loaded successfully!")
print("Historical Stocks shape:", historical_stocks.shape)
print("Historical Stock Prices shape:", historical_stock_prices.shape)

Dataset loaded successfully!
Historical Stocks shape: (6460, 5)
Historical Stock Prices shape: (20973889, 8)


In [ ]:
# Step 1: Advanced Data Cleaning - Handle Missing Values
print("=== ADVANCED DATA CLEANING ===")
print("\
1. Missing Values Analysis:")

# Check missing values in both datasets
print("Historical Stocks missing values:")
stocks_missing = historical_stocks.isnull().sum()
print(stocks_missing)

print("\
Historical Stock Prices missing values:")
prices_missing = historical_stock_prices.isnull().sum()
print(prices_missing)

# Handle missing values in historical_stocks
print("\
2. Handling Missing Values:")
print("Before cleaning - Stocks dataset shape:", historical_stocks.shape)

# Fill missing sector and industry with 'Unknown'
historical_stocks['sector'] = historical_stocks['sector'].fillna('Unknown')
historical_stocks['industry'] = historical_stocks['industry'].fillna('Unknown')

print("After filling missing values - Stocks dataset:")
print(historical_stocks.isnull().sum())

# Convert date column to datetime
historical_stock_prices['date'] = pd.to_datetime(historical_stock_prices['date'])
print("\
Date conversion completed successfully")

=== ADVANCED DATA CLEANING ===
1. Missing Values Analysis:
Historical Stocks missing values:
ticker         0
exchange       0
name           0
sector      1440
industry    1440
dtype: int64
Historical Stock Prices missing values:
ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
date         0
dtype: int64
2. Handling Missing Values:
Before cleaning - Stocks dataset shape: (6460, 5)
After filling missing values - Stocks dataset:
ticker      0
exchange    0
name        0
sector      0
industry    0
dtype: int64
Date conversion completed successfully


In [ ]:
# Step 2: Outlier Detection and Analysis
print("3. Outlier Detection and Analysis:")

# Focus on key numerical columns for outlier detection
numerical_cols = ['open', 'close', 'adj_close', 'low', 'high', 'volume']

# Calculate basic statistics
print("Basic statistics for numerical columns:")
stats_summary = historical_stock_prices[numerical_cols].describe()
print(stats_summary)

# Detect outliers using IQR method for close prices and volume
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# Detect outliers in close prices
close_outliers, close_lower, close_upper = detect_outliers_iqr(historical_stock_prices, 'close')
print(f"\
Close price outliers: {len(close_outliers)} records")
print(f"Close price bounds: [{close_lower:.2f}, {close_upper:.2f}]")

# Detect outliers in volume
volume_outliers, vol_lower, vol_upper = detect_outliers_iqr(historical_stock_prices, 'volume')
print(f"Volume outliers: {len(volume_outliers)} records")
print(f"Volume bounds: [{vol_lower:.0f}, {vol_upper:.0f}]")

3. Outlier Detection and Analysis:
Basic statistics for numerical columns:
               open         close     adj_close           low          high  \
count  2.097389e+07  2.097389e+07  2.097389e+07  2.097389e+07  2.097389e+07   
mean   7.605823e+01  7.611403e+01  1.481184e+14  7.422064e+01  7.803857e+01   
std    2.849639e+03  2.870159e+03  4.574674e+16  2.746059e+03  2.997937e+03   
min    4.000000e-04  2.000000e-04  2.282650e-09  1.000000e-04  4.000000e-04   
25%    7.500000e+00  7.500000e+00  4.620000e+00  7.360000e+00  7.630000e+00   
50%    1.545000e+01  1.545000e+01  1.138199e+01  1.524000e+01  1.566000e+01   
75%    2.972000e+01  2.972000e+01  2.472046e+01  2.928000e+01  3.010000e+01   
max    2.034000e+06  1.779750e+06  1.894962e+19  1.440000e+06  2.070000e+06   

             volume  
count  2.097389e+07  
mean   1.227043e+06  
std    1.316686e+07  
min    1.000000e+00  
25%    2.210000e+04  
50%    1.260000e+05  
75%    6.074000e+05  
max    4.483504e+09  
Close price out

In [ ]:
# Investigate the extreme values more carefully
print("4. Detailed Outlier Investigation:")

# Check for impossible values (negative prices, zero volume patterns)
print("Data quality checks:")
print(f"Negative open prices: {(historical_stock_prices['open'] < 0).sum()}")
print(f"Negative close prices: {(historical_stock_prices['close'] < 0).sum()}")
print(f"Negative high prices: {(historical_stock_prices['high'] < 0).sum()}")
print(f"Negative low prices: {(historical_stock_prices['low'] < 0).sum()}")
print(f"Zero volume records: {(historical_stock_prices['volume'] == 0).sum()}")

# Check for logical inconsistencies (high < low, etc.)
logical_errors = historical_stock_prices[
    (historical_stock_prices['high'] < historical_stock_prices['low']) |
    (historical_stock_prices['close'] > historical_stock_prices['high']) |
    (historical_stock_prices['close'] < historical_stock_prices['low']) |
    (historical_stock_prices['open'] > historical_stock_prices['high']) |
    (historical_stock_prices['open'] < historical_stock_prices['low'])
]
print(f"Logical inconsistencies: {len(logical_errors)} records")

# Check the adj_close column which seems to have extreme values
print(f"\
Adjusted close statistics:")
print(f"Min: {historical_stock_prices['adj_close'].min()}")
print(f"Max: {historical_stock_prices['adj_close'].max()}")
print(f"Mean: {historical_stock_prices['adj_close'].mean()}")

# Sample some extreme adj_close values
extreme_adj = historical_stock_prices.nlargest(10, 'adj_close')[['ticker', 'date', 'close', 'adj_close']]
print("\
Top 10 extreme adjusted close values:")
print(extreme_adj)

4. Detailed Outlier Investigation:
Data quality checks:
Negative open prices: 0
Negative close prices: 0
Negative high prices: 0
Negative low prices: 0
Zero volume records: 0
Logical inconsistencies: 2076 records
Adjusted close statistics:
Min: 2.2826502910078303e-09
Max: 1.89496189108124e+19
Mean: 148118439358479.7
Top 10 extreme adjusted close values:
        ticker       date     close     adj_close
6174033    AAN 1987-02-27  1.296296  1.894962e+19
6174039    AAN 1987-03-10  1.296296  1.894962e+19
6174040    AAN 1987-03-11  1.296296  1.894962e+19
6174041    AAN 1987-03-12  1.296296  1.894962e+19
6174044    AAN 1987-03-17  1.296296  1.894962e+19
6174045    AAN 1987-03-18  1.296296  1.894962e+19
6174046    AAN 1987-03-19  1.296296  1.894962e+19
6174049    AAN 1987-03-25  1.296296  1.894962e+19
6174051    AAN 1987-03-27  1.287037  1.881428e+19
6174050    AAN 1987-03-26  1.268519  1.854356e+19


In [ ]:
# Step 3: Data Cleaning and Error Correction
print("5. Data Cleaning and Error Correction:")

# Create a copy for cleaning
cleaned_prices = historical_stock_prices.copy()

# Remove logical inconsistencies
print(f"Before removing logical errors: {len(cleaned_prices)} records")
cleaned_prices = cleaned_prices[
    (cleaned_prices['high'] >= cleaned_prices['low']) &
    (cleaned_prices['close'] <= cleaned_prices['high']) &
    (cleaned_prices['close'] >= cleaned_prices['low']) &
    (cleaned_prices['open'] <= cleaned_prices['high']) &
    (cleaned_prices['open'] >= cleaned_prices['low'])
]
print(f"After removing logical errors: {len(cleaned_prices)} records")

# Handle extreme adjusted close values - cap at reasonable bounds
# Calculate reasonable bounds based on close prices
adj_close_q99 = cleaned_prices['close'].quantile(0.99)
adj_close_q01 = cleaned_prices['close'].quantile(0.01)

print(f"Close price 99th percentile: {adj_close_q99:.2f}")
print(f"Close price 1st percentile: {adj_close_q01:.2f}")

# Cap adjusted close values to reasonable bounds
cleaned_prices['adj_close'] = np.where(
    cleaned_prices['adj_close'] > adj_close_q99 * 10,  # Allow 10x the 99th percentile
    cleaned_prices['close'],  # Replace with close price
    cleaned_prices['adj_close']
)

cleaned_prices['adj_close'] = np.where(
    cleaned_prices['adj_close'] < adj_close_q01 / 10,  # Allow 1/10th the 1st percentile
    cleaned_prices['close'],  # Replace with close price
    cleaned_prices['adj_close']
)

print(f"After capping extreme adj_close values:")
print(f"New adj_close max: {cleaned_prices['adj_close'].max():.2f}")
print(f"New adj_close min: {cleaned_prices['adj_close'].min():.2f}")

# Remove duplicate records
print(f"\
Before removing duplicates: {len(cleaned_prices)} records")
cleaned_prices = cleaned_prices.drop_duplicates(subset=['ticker', 'date'])
print(f"After removing duplicates: {len(cleaned_prices)} records")

5. Data Cleaning and Error Correction:
Before removing logical errors: 20973889 records
After removing logical errors: 20971813 records
Close price 99th percentile: 282.00
Close price 1st percentile: 0.59
After capping extreme adj_close values:
New adj_close max: 1779750.00
New adj_close min: 0.00
Before removing duplicates: 20971813 records
After removing duplicates: 20971813 records


In [ ]:
# Step 4: Feature Engineering
print("=== DATA TRANSFORMATION ===")
print("\
6. Feature Engineering:")

# Sort data by ticker and date for time series calculations
cleaned_prices = cleaned_prices.sort_values(['ticker', 'date'])

# Calculate rolling averages (5, 10, 20, 50 day moving averages)
print("Creating rolling averages...")
for window in [5, 10, 20, 50]:
    cleaned_prices[f'ma_{window}'] = cleaned_prices.groupby('ticker')['close'].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )

# Calculate volatility measures
print("Creating volatility measures...")
cleaned_prices['daily_return'] = cleaned_prices.groupby('ticker')['close'].pct_change()
cleaned_prices['volatility_10d'] = cleaned_prices.groupby('ticker')['daily_return'].transform(
    lambda x: x.rolling(window=10, min_periods=1).std()
)
cleaned_prices['volatility_30d'] = cleaned_prices.groupby('ticker')['daily_return'].transform(
    lambda x: x.rolling(window=30, min_periods=1).std()
)

# Calculate technical indicators
print("Creating technical indicators...")

# RSI (Relative Strength Index) - simplified version
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window, min_periods=1).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

cleaned_prices['rsi'] = cleaned_prices.groupby('ticker')['close'].transform(calculate_rsi)

# Price momentum (price change over different periods)
for period in [1, 5, 10, 20]:
    cleaned_prices[f'momentum_{period}d'] = cleaned_prices.groupby('ticker')['close'].transform(
        lambda x: x.pct_change(periods=period)
    )

# Volume indicators
cleaned_prices['volume_ma_10'] = cleaned_prices.groupby('ticker')['volume'].transform(
    lambda x: x.rolling(window=10, min_periods=1).mean()
)
cleaned_prices['volume_ratio'] = cleaned_prices['volume'] / cleaned_prices['volume_ma_10']

print(f"Feature engineering completed. New shape: {cleaned_prices.shape}")
print(f"New features created: {cleaned_prices.shape[1] - 8} additional columns")

=== DATA TRANSFORMATION ===
6. Feature Engineering:
Creating rolling averages...
Creating volatility measures...
Creating technical indicators...
Feature engineering completed. New shape: (20971813, 22)
New features created: 14 additional columns
